### 1. Libraries

In [18]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
from sklearn.model_selection import train_test_split
import sys
import os
import numpy as np
import math
import random, time

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt
import gc 
import glob

ModuleNotFoundError: No module named 'sklearn'

### 2. Load directory

In [2]:
# Main directory 
mainPath = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5"

directory= os.path.abspath(mainPath)

# Add directory to sys.path
sys.path.append(directory)

# import script where parameters are defined
import param  

### 3. Defining parameters

In [3]:
# Vizualizancing main directories 
list_of_paths = param.list_paths 

# Defining amount of train,valid,test images   
n_train=param.n_train;    n_valid=param.n_valid;    n_test=param.n_test

# Defining a number of samples
n_sample=param.n_sample

# For resizing dimentions
h=param.img_height
w=param.img_width

#Pack them into batches (we’ll use batches of 10 images e.g.)
BATCH_SIZE=param.pack_size 

'''
Showing parameters
'''
for i,j in enumerate(list_of_paths):
    print(f'{i} => {j}')

print(f'\nTrain = {n_train}, Test = {n_test}, Valid = {n_valid}\n')
print(f'Dimensions resized h = {h}, w = {w}\n')
print(f'Pack into {BATCH_SIZE} images')    

0 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Geometry
1 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Magnitude
2 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Pression
3 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U001
4 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U002

Train = 0.8, Test = 0.2, Valid = 0.0

Dimensions resized h = 128, w = 256

Pack into 100 images


In [ ]:
def process_images_in_batches(img_dir, batch_size=500, target_shape=(128, 128)):
    filenames = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    n_files = len(filenames)
    
    for start in range(0, n_files, batch_size):
        end = min(start + batch_size, n_files)
        batch_files = filenames[start:end]
        
        batch_data = []
        for fname in batch_files:
            img = imread(os.path.join(img_dir, fname))
            img = resize(img, target_shape, anti_aliasing=True)
            batch_data.append(img)
        
        batch_array = np.array(batch_data)
        
        # Aquí puedes aplicar lo que necesites al batch
        print(f"Procesado batch {start}-{end} -> shape: {batch_array.shape}")
        
        # Libera memoria después de procesar cada batch
        del batch_data, batch_array
        gc.collect()


In [12]:
  
number_batch=500    

In [14]:
min(80,70)

70

In [17]:
#  geo_train,geo_valid,geo_tests=split_data(list_of_paths[0],n_sample,n_train,n_valid,n_test) 
#  geo_array_train=arrays_img_processed(geo_train)
def  process_by_batches(list_of_path,number_batch,option=True):
    files=sorted([os.path.join(list_of_paths,fname) for fname in os.listdir(list_of_paths) if fname.endswith(".png")])
    n_files=len(files)
    for start in range(0,n_files,number_batch):
         data_batch=[]
         end = min(start + number_batch, n_files)
         file_batches=files[start:end]
         for img in file_batches:
             x = processing(img,option)
             data_batch.append(x)
         del data_batch
         gc.collect()
             
         
    

### 4. Process data

In [ ]:
#method for reading image    
def get_img(img_name):
    return ski.io.imread(img_name)

# method for turning to a grey or binary image 
def processing(image,option=True):
        x = get_img(image)  
        x = rgb2gray(x)       # It returns a grayscale image with floating point values in the range from 0 to 1
        x =cv.resize(x,(w,h)) # Reshape image 
    
        # Binary option otherwise gray
        if (option):
            x=ski.util.img_as_ubyte(x)  # Convert it back to the original data type and the data range back 0 to 255. 
                                        # It is often better to use image values represented by floating point values 
            best_value_threshold=np.round(filters.threshold_otsu(x)) #  Otsu’s method calculates an “optimal” threshold

            _,x= cv.threshold(x, best_value_threshold, 255, cv.THRESH_BINARY)
            x=x/255.
        x=x[:, :, np.newaxis]
        return x

def arrays_img_processed(data,type=True): # True for Geometry 
    # begin in data[1:] 
    if type:
        list_arrays=[processing(image,option=True).astype(np.float32) for image in data[1:] ]
    else:
        list_arrays=[processing(image,option=False).astype(np.float32) for image in data[1:] ]
   
    return np.array(list_arrays) 
    
# Create a mask
def create_mask(u,v):
    return u*v

# method for splitting data
def split_data(path,n_sample,n_train,n_valid,n_test):
        
        files=sorted([os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith(".png")])
        n_files=len(files)
        
        if(n_files>=n_sample):
                print('Loaded ',n_sample,'/',n_files,'**** from ',path,'\n')
                if n_valid == 0.:
                    if (n_train + n_test == 1.0):
                        n_train= math.floor(n_sample*n_train)
                        n_test = math.floor(n_sample*n_test) + n_train
                        (files_train,files_tests) = (files[0:n_train], files[n_train:n_test])
                        return files_train,files_tests
    
                else:     
                    if (n_train + n_valid + n_test == 1.0):
        
                        n_train= math.floor(n_sample*n_train)
                        n_valid = math.floor(n_sample*n_valid) + n_train
                        n_test = math.floor(n_sample*n_test) + n_valid
              
                        (files_train,files_valid,files_tests) = (files[0:n_train], files[n_train:n_valid],files[n_valid:n_test])
                    
                        return files_train,files_valid,files_tests

    

#### 4.1 Processing

#### 4.2 Dataset Objects

#### 4.2 Create a dataset object

In [ ]:
if n_valid != 0.:    
    
        geo_train,geo_valid,geo_tests=split_data(list_of_paths[0],n_sample,n_train,n_valid,n_test)    # Geometry
        mag_train,mag_valid,mag_tests=split_data(list_of_paths[1],n_sample,n_train,n_valid,n_test)   # Magnitude
        p_train,p_valid,p_tests=split_data(list_of_paths[2],n_sample,n_train,n_valid,n_test)        # Pression
        vx_train,vx_valid,vx_tests=split_data(list_of_paths[3],n_sample,n_train,n_valid,n_test)    # Vx
        vy_train,vy_valid,vy_tests=split_data(list_of_paths[4],n_sample,n_train,n_valid,n_test)   # Vy

        geo_array_train=arrays_img_processed(geo_train)
        geo_array_valid=arrays_img_processed(geo_valid)
        geo_array_tests=arrays_img_processed(geo_tests)

        mag_array_train=arrays_img_processed(mag_train,False)
        mag_array_valid=arrays_img_processed(mag_valid,False)
        mag_array_tests=arrays_img_processed(mag_tests,False)

        p_array_train=arrays_img_processed(p_train,False)
        p_array_valid=arrays_img_processed(p_valid,False)
        p_array_tests=arrays_img_processed(p_tests,False)

        vx_array_train=arrays_img_processed(vx_train,False)
        vx_array_valid=arrays_img_processed(vx_valid,False)
        vx_array_tests=arrays_img_processed(vx_tests,False)

        vy_array_train=arrays_img_processed(vy_train,False)
        vy_array_valid=arrays_img_processed(vy_valid,False)
        vy_array_tests=arrays_img_processed(vy_tests,False)

        mag_masked_train = create_mask(geo_array_train,mag_array_train)
        mag_masked_valid = create_mask(geo_array_valid,mag_array_valid)
        mag_masked_test  = create_mask(geo_array_test,mag_array_test)

        p_masked_train = create_mask(geo_array_train,p_array_train)
        p_masked_valid = create_mask(geo_array_valid,p_array_valid)
        p_masked_test =  create_mask(geo_array_test,p_array_test)

        vx_masked_train = create_mask(geo_array_train,vx_array_train)
        vx_masked_valid = create_mask(geo_array_train,vx_array_train)
        vx_masked_test =  create_mask(geo_array_train,vx_array_train)

        vy_masked_train = create_mask(geo_array_train,vy_array_train)
        vy_masked_valid = create_mask(geo_array_train,vy_array_train)
        vy_masked_test =  create_mask(geo_array_train,vy_array_train)

else:
        geo_train, geo_tests=split_data(list_of_paths[0],n_sample,n_train,n_valid,n_test)
        mag_train,mag_tests=split_data(list_of_paths[1],n_sample,n_train,n_valid,n_test)
        p_train,p_tests=split_data(list_of_paths[2],n_sample,n_train,n_valid,n_test)
        vx_train,vx_tests=split_data(list_of_paths[3],n_sample,n_train,n_valid,n_test)
        vy_train,vy_tests=split_data(list_of_paths[4],n_sample,n_train,n_valid,n_test)

        geo_array_train=arrays_img_processed(geo_train)
        geo_array_tests=arrays_img_processed(geo_tests)

        mag_array_train=arrays_img_processed(mag_train,False)
        mag_array_tests=arrays_img_processed(mag_tests,False)

        p_array_train=arrays_img_processed(p_train,False)
        p_array_tests=arrays_img_processed(p_tests,False)

        vx_array_train=arrays_img_processed(vx_train,False)
        vx_array_tests=arrays_img_processed(vx_tests,False)

        vy_array_train=arrays_img_processed(vy_train,False)
        vy_array_tests=arrays_img_processed(vy_tests,False)
        # *******************************************************************
        mag_masked_train = create_mask(geo_array_train,mag_array_train)
        mag_masked_test  = create_mask(geo_array_tests,mag_array_tests)

        p_masked_train = create_mask(geo_array_train,p_array_train)
        p_masked_test =  create_mask(geo_array_tests,p_array_tests)

        vx_masked_train = create_mask(geo_array_train,vx_array_train)
        vx_masked_test =  create_mask(geo_array_tests,vx_array_tests)
    
        vy_masked_train = create_mask(geo_array_train,vy_array_train)
        vy_masked_test =  create_mask(geo_array_tests,vy_array_tests)

print(f'Training data for geometry: {geo_array_train.shape}')
print(f'Test data for geometry: {geo_array_tests.shape}')


In [ ]:
# Create a tensor of four channels
data_merged_train = np.concatenate((mag_masked_train, p_masked_train, vx_masked_train, vy_masked_train), axis=-1)
data_merged_tests = np.concatenate((mag_masked_test,p_masked_test,vx_masked_test, vy_masked_test), axis=-1)

# slices of an array in the form of objects by using tf.data.Dataset.from_tensor_slices() method
train_data=tf.data.Dataset.from_tensor_slices((geo_array_train, data_merged_train))
tests_data=tf.data.Dataset.from_tensor_slices((geo_array_tests, data_merged_tests))

print(data_merged_train.shape)
print(data_merged_tests.shape)

#### 4.3 Visualizing dataset 

In [ ]:
def show_image(img1,img2,img3,img4,title=None,opt=None,prediction=None):
    if opt==None and prediction==None:
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 5), sharey=True)
        axes[0, 0].set_xticks([]);axes[0, 0].set_yticks([]);
        axes[1, 0].set_xticks([]); axes[1, 0].set_yticks([]);
        axes[0, 1].set_xticks([]); axes[0, 1].set_yticks([]);
        axes[1, 1].set_xticks([]); axes[1, 1].set_yticks([]);
        
        axes[0, 0].imshow(img1,origin='upper',cmap='gray')
        axes[0, 0].set_title('Input',fontsize=14)
        axes[0, 0].set_ylabel('Geometry',fontsize=14)

        axes[0, 1].imshow(img2,origin='upper',cmap='gray')
        axes[0, 1].set_title('Output-Magnitude',fontsize=14)
        axes[0, 1].set_ylabel('V',fontsize=14)

        axes[1, 0].imshow(img3,origin='upper',cmap='gray')
        axes[1, 0].set_title('Output x-velocity',fontsize=14)
        axes[1, 0].set_ylabel('Vx',fontsize=14)

        axes[1, 1].imshow(img4,origin='upper',cmap='gray')
        axes[1, 1].set_title('Output-Pression',fontsize=14)
        axes[1, 1].set_ylabel('P',fontsize=14)

        if title is not None: fig.suptitle(title)
    
    else:
        fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(16, 5), sharey=True)
        ax1.set_xticks([]); ax1.set_yticks([]);
        ax2.set_xticks([]); ax2.set_yticks([]);
        ax1.imshow(a1,origin='upper',cmap='gray')
        ax1.set_title('Real',fontsize=14)
        #ax1.set_xlabel('Geometry')
        ax1.set_ylabel('u',fontsize=14)
        ax2.imshow(a2,origin='upper',cmap='gray')
        ax2.set_title('Predicted',fontsize=14)
        #ax2.set_xlabel('u')
        ax2.set_ylabel(prediction,fontsize=16)
        if title is not None: fig.suptitle(title)
        
        

In [ ]:
img_random=random.sample(range(1, geo_array_train.shape[0]), 1)
show_image(geo_array_train[img_random[0]],mag_masked_train[img_random[0]],
           vx_masked_train[img_random[0]],p_masked_train[img_random[0]],
           title=f"Training data sample N°{img_random[0]}")

### 5. Setup Hiperparameters

In [ ]:
num_classes =param.num_classes
num_epochs =param.num_epochs
batch_size=param.batch_size
patience=param.patience
LR=param.LR
print(f'Classes= {num_classes},\n Epochs = {num_epochs},\n batches = {batch_size},\n lr = {LR}')

### 6. Architecture

In [ ]:
# construct model
inputs = Input(train_geometries.shape[1:])